## IBM Quantum Challenge Fall 2021
# Challenge 2: OLED 분자들의 밴드갭 계산

<div id='problem'></div>
<div class="alert alert-block alert-info">
    
최고의 경험을 위해 오른쪽 상단의 계정 메뉴에서 **light** 워크스페이스 테마로 전환하는 것을 추천합니다.</div>

## 소개

유기 발광 다이오드(Organic Light Emitting Diode) 또는 OLED는 전류를 인가하면 빛을 내는, 얇고 유연한 TV 및 휴대폰 디스플레이 제조의 기초 소자로 최근 몇 년 동안 점점 인기를 얻고 있습니다.
최근 연구([**Gao et al., 2021**](https://www.nature.com/articles/s41524-021-00540-6))에서는 페닐설포닐-카바졸(PSPCz) 분자에서 고에너지 상태의 전자 전이를 관측했으며, 이는 OLED 기술에 유용한 열 활성화 지연 형광(TADF) 이미터에 활용 될 수있습니다. OLED를 만들기 위해 현재 사용되는 양자 효율이 25%로 제한된 기존의 형광 포어와 비교하여 TADF 방출기는 100% 내부 양자 효율(IQP)-흡수된 광자를 방출하는 회로 또는 시스템에서 전하 캐리어의 비율-로 작동하는 OLED를 생산할 가능성을 지니고 있습니다. 효율성의 큰 증가는 제조업체들이 휴대폰 처럼 낮은 소비 전력을 요구하는 기기에 사용할 OLED를 생산할 수 있다는 것을 의미하며, 이는 결과적으로 가정, 사무실, 박물관 그리고 그보다 더 많은 넓은 면적의 공간들이 값싸고 에너지 효율이 높은 조명원을 사용하게 되는 미래의 개발로 이어질 수 있음을 의미합니다!

<center><img src="resources/JSR_img6_1920w.jpg" width="600"></center>

### Why quantum?

양자 컴퓨터는 고전적인 컴퓨터보다 양자 장치에서 양자 역학 시스템을 모델링하는 것이 지닌 자연스러움으로 인해 복잡한 분자 및 재료의 전자 구조 및 동적 특성을 연구하는 데 귀중한 도구가 될 것으로 기대되고 있습니다. IBM Quantum과 파트너의 최근 공동 연구 프로젝트는 효율적인 OLED를 위한 TADF의 들뜬 상태 계산의 정확도를 향상시키는 방법을 성공적으로 개발하여 상용 재료의 들뜬 상태 계산에 양자 컴퓨터를 적용한 세계 최초의 연구 사례가 되었습니다(위에 링크된 논문 참조).
    

이러한 배경 정보와 함께, 효율적인 유기발광다이오드(OLED) 소자의 제작에 잠재적으로 사용될 수 있는, 산업적 화학 화합물의 "들뜬 상태" 또는 높은 에너지 상태에 대한 양자 계산을 설명하고자 합니다.

## 도전 과제

<div class="alert alert-block alert-success">

<b>목표</b> 

이 도전의 목표는 양자 알고리즘을 사용하여 TADF 물질의 들뜬 상태 에너지를 신뢰성 있게 계산하는 것입니다. 그러기 위해서 이 노트북을 통해 고전적 근사 단계와 보다 정확한 양자 계산 단계 간의 작업 부하 분할을 허용하는 최첨단 하이브리드 고전-양자 방식의 화학 모델링을 도입합니다.
    
1. **도전 2a & 2b**: 원자 궤도(AO), 분자 궤도 (MO), 능동 공간 변환(Active Space Transformation)을 사용하여 궤도의 수를 줄이는 방법을 이해합니다.
2. **도전 2c & 2d**: NumPy 및 VQE(Variational Quantum Eigensolver)를 사용하여 PSPCz 분자의 바닥 상태 에너지를 계산합니다.
3. **도전 2e**: 양자 운동 방정식(QEOM) 알고리즘을 사용하여 PSPCz 분자의 들뜬 상태 에너지를 계산합니다.
4. **도전 2f**: Qiskit Runtime을 사용하여 클라우드(시뮬레이터 또는 실제 양자 시스템)에서 VQE를 실행합니다.
</div>

<div class="alert alert-block alert-info">

시작하기에 앞서,[**Qiskit Nature Demo Session with Max Rossmannek**](https://youtu.be/UtMVoGXlz04?t=38)을 시청하고 데모에 사용된 [**demo notebook**](https://github.com/qiskit-community/qiskit-application-modules-demo-sessions/tree/main/qiskit-nature)을 통해 전자 구조 계산을 미리 배워보길 추천합니다.

</div>

### 1. 드라이버(Driver)

Qiskit과 고전적인 화학 코드 사이의 인터페이스를 드라이버라고 합니다. `PSI4Driver`, `PyQuanteDriver`, `PySCFDriver` 등이 있습니다.

아래의 셀에서 드라이버(주어진 기저 집합과 분자의 구조 정보에 대한 하트리-폭 계산)를 실행하여, 양자 알고리즘을 적용하기 위해 알아야 하는 대상 분자에 대한 모든 필요한 정보를 얻습니다.

In [ ]:
from qiskit_nature.drivers import Molecule
from qiskit_nature.drivers.second_quantization import ElectronicStructureDriverType, ElectronicStructureMoleculeDriver

# PSPCz molecule
geometry = [['C', [  -0.2316640,    1.1348450,    0.6956120]],
            ['C', [  -0.8886300,    0.3253780,   -0.2344140]],
            ['C', [  -0.1842470,   -0.1935670,   -1.3239330]],
            ['C', [   1.1662930,    0.0801450,   -1.4737160]],
            ['C', [   1.8089230,    0.8832220,   -0.5383540]],
            ['C', [   1.1155860,    1.4218050,    0.5392780]],
            ['S', [   3.5450920,    1.2449890,   -0.7349240]],
            ['O', [   3.8606900,    1.0881590,   -2.1541690]],
            ['C', [   4.3889120,   -0.0620730,    0.1436780]],
            ['O', [   3.8088290,    2.4916780,   -0.0174650]],
            ['C', [   4.6830900,    0.1064460,    1.4918230]],
            ['C', [   5.3364470,   -0.9144080,    2.1705280]],
            ['C', [   5.6895490,   -2.0818670,    1.5007820]],
            ['C', [   5.4000540,   -2.2323130,    0.1481350]],
            ['C', [   4.7467230,   -1.2180160,   -0.5404770]],
            ['N', [  -2.2589180,    0.0399120,   -0.0793330]],
            ['C', [  -2.8394600,   -1.2343990,   -0.1494160]],
            ['C', [  -4.2635450,   -1.0769890,    0.0660760]],
            ['C', [  -4.5212550,    0.2638010,    0.2662190]],
            ['C', [  -3.2669630,    0.9823890,    0.1722720]],
            ['C', [  -2.2678900,   -2.4598950,   -0.3287380]],
            ['C', [  -3.1299420,   -3.6058560,   -0.3236210]],
            ['C', [  -4.5179520,   -3.4797390,   -0.1395160]],
            ['C', [  -5.1056310,   -2.2512990,    0.0536940]],
            ['C', [  -5.7352450,    1.0074800,    0.5140960]],
            ['C', [  -5.6563790,    2.3761270,    0.6274610]],
            ['C', [  -4.4287740,    3.0501460,    0.5083650]],
            ['C', [  -3.2040560,    2.3409470,    0.2746950]],
            ['H', [  -0.7813570,    1.5286610,    1.5426490]],
            ['H', [  -0.7079140,   -0.7911480,   -2.0611600]],
            ['H', [   1.7161320,   -0.2933710,   -2.3302930]],
            ['H', [   1.6308220,    2.0660550,    1.2427990]],
            ['H', [   4.4214900,    1.0345500,    1.9875450]],
            ['H', [   5.5773000,   -0.7951290,    3.2218590]],
            ['H', [   6.2017810,   -2.8762260,    2.0345740]],
            ['H', [   5.6906680,   -3.1381740,   -0.3739110]],
            ['H', [   4.5337010,   -1.3031330,   -1.6001680]],
            ['H', [  -1.1998460,   -2.5827750,   -0.4596910]],
            ['H', [  -2.6937370,   -4.5881470,   -0.4657540]],
            ['H', [  -5.1332290,   -4.3740010,   -0.1501080]],
            ['H', [  -6.1752900,   -2.1516170,    0.1987120]],
            ['H', [  -6.6812260,    0.4853900,    0.6017680]],
            ['H', [  -6.5574610,    2.9529350,    0.8109620]],
            ['H', [  -4.3980410,    4.1305040,    0.5929440]],
            ['H', [  -2.2726630,    2.8838620,    0.1712760]]]

molecule = Molecule(geometry=geometry, charge=0, multiplicity=1)

driver = ElectronicStructureMoleculeDriver(molecule=molecule, 
                                           basis='631g*', 
                                           driver_type=ElectronicStructureDriverType.PYSCF)

<div class="alert alert-block alert-success">
    
<b> 도전 2a</b> 
    
질문: PSPCz 분자의 다음의 값들을 찾아봅시다.
    
1. C, H, N, O, S 원자들은 각각 몇개입니까?
1. 원자들의 총 몇 개입니까?
1. 원자 궤도는 전부 몇개입니까 (AO) ? 
1. 분자 궤도는 전부 몇개입니까 (MO) ?

</div>

<div class="alert alert-block alert-danger">
    
**원자의 궤도를 어떻게 셀 수 있나요?**

궤도의 숫자는 베이시스와 관련이 있습니다. 아래의 숫자는 이 도전에 사용될 `631g*` 베이시스의 숫자들입니다.

- C: 1s, 2s2p, 3s3p3d  = 1+4+9 = 14

- H: 1s, 2s  = 1+1 = 2

- N: 1s, 2s2p, 3s3p3d  = 1+4+9 = 14

- O: 1s, 2s2p, 3s3p3d  = 1+4+9 = 14

- S: 1s, 2s2p, 3s3p3d, 4s4p = 1+4+9+4 = 18</div>

In [ ]:
num_ao = {
    'C': 14,
    'H': 2,
    'N': 14,
    'O': 14,
    'S': 18,
}

##############################
# Provide your code here

num_C_atom = 
num_H_atom = 
num_N_atom =
num_O_atom =
num_S_atom =
num_atoms_total = 
num_AO_total = 
num_MO_total = 

##############################

answer_ex2a ={
    'C': num_C_atom,
    'H': num_H_atom,
    'N': num_N_atom,
    'O': num_O_atom,
    'S': num_S_atom,
    'atoms': num_atoms_total,
    'AOs': num_AO_total,
    'MOs': num_MO_total
}

print(answer_ex2a)

In [ ]:
# Check your answer and submit using the following code
from qc_grader import grade_ex2a
grade_ex2a(answer_ex2a)

위의 연습에서 발견했듯이 PSPCz는 많은 원자와 많은 원자 궤도로 구성된 거대한 분자입니다. 거대한 분자는 현재의 양자 시스템으로 직접 계산하기 어렵습니다. 그러나 이 도전에서는 밴드갭에만 관심이 있으므로 HOMO(Highest Occuppied Molecular Orbital) 및 LUMO(Lowest Unoccuppied Molecular Orbital)의 에너지를 계산하는 것만으로도 충분합니다. 이 부분에서는 분자 궤도의 수를 2개(HOMO 및 LUMO)로 줄이기 위해 능동 공간 변환(Active Space Transformation) 기술을 사용해 보도록 하겠습니다:

$$E_g = E_{LUMO} - E_{HOMO}$$


<center><img src="resources/Molecule_HOMO-LUMO_diagram.svg" width="600"></center>


이 그림의 동그라미들은 궤도에 있는 전자입니다; HOMO에 위치한 전자가 충분히 높은 주파수의 에너지나 빛을 흡수하면 전자는 LUMO 상태로 점프하여 올라갑니다. 

이번 PSPCz 분자의 문제에서 들뜬 상태는 첫 번째 단일항(singlet) 및 삼중항(triplet) 상태로 제한합니다. 단일항 상태에서 시스템의 모든 전자는 스핀 쌍을 이루며 공간에서 가능한 한 방향 선택할 수 있습니다. 단일 또는 삼중항 상태는 두 전자 중 하나를 더 높은 에너지 수준으로 들뜨게하여 생성됩니다. 들뜬 전자는 단일 들뜬 상태에서 동일한 스핀 방향을 유지하는 반면, 삼중 들뜬 상태에서 들뜬 전자는 바닥 상태 전자와 동일한 스핀 방향을 가집니다.


<center><img src="resources/spin.jpg" width="300"><figcaption>Spin in the ground and excited states</figcaption></center>

삼중항 상태에서 전자 스핀의 한 세트는 짝을 이루지 않습니다. 즉, 공간의 세 방향 축을 스핀이 선택 할 수 있습니다. PSPCz (a, d)와 그 변형 2F-PSPCz (b, e),  4F-PSPCz (c, f)의 삼중항 상태에서 최적화된 구조의 LUMO (a-c)와 HOMO (e-f)는 다음 그림과 같을 것입니다.

<center><img src="resources/oled_paper_fig2.jpg" width="600"></center>

<center><img src="resources/oled_paper_fig1.jpg" width="600"></center>


능동 공간 변환 방법을 사용하여, 가능한 가장 작은 활성 공간인 단일항와 삼중항으로 계산을 제한함과 동시에 비핵심 전자 상태를 배제하면서 시스템에 대한 높은 수준의 표현력을 유지하면서 적은 수의 큐비트로 이 에너지를 계산할 수 있습니다.

In [ ]:
from qiskit_nature.drivers.second_quantization import HDF5Driver

driver_reduced = HDF5Driver("resources/PSPCz_reduced.hdf5")
properties = driver_reduced.run()

In [ ]:
from qiskit_nature.properties.second_quantization.electronic import ElectronicEnergy

electronic_energy = properties.get_property(ElectronicEnergy)
print(electronic_energy)

`(AO) 1-Body Terms`은 대상 분자가 가진 전체 430개의 분자 궤도에 대한 430개의 원자 궤도를 표현하는 (430 x 430) 행렬을 지니고 있습니다. `ActiveSpaceTransformation`(미리 계산되어 있음) 이후에, 분자 궤도 `(MO) 1-Body Terms`의 수는 (2x2) 행렬로 감소합니다.

<div class="alert alert-block alert-success">
    
<b> 도전 2b</b> 
    
질문: 속성 프레임워크(Property framework)를 사용하여 아래 질문에 대한 답하시오.
    
1. 능동 공간 변환 후 시스템의 전자 수는 얼마입니까?
1. 분자 궤도는 몇 개입니까? What is the number of molecular orbitals?
1. Spin-orbital은 몇 개입니까? What is the number of spin-orbitals?
1. Jordan-Wigner 매핑으로 이 분자를 시뮬레이션하려면 몇 개의 큐비트가 필요합니까?

</div>

In [ ]:
from qiskit_nature.properties.second_quantization.electronic import ParticleNumber

##############################
# Provide your code here

particle_number = 
num_electron = 
num_MO = 
num_SO = 
num_qubits = 

##############################

answer_ex2b = {
    'electrons': num_electron,
    'MOs': num_MO,
    'SOs': num_SO,
    'qubits': num_qubits
}

print(answer_ex2b)

In [ ]:
# Check your answer and submit using the following code
from qc_grader import grade_ex2b

grade_ex2b(answer_ex2b)

### 2. 전자 구조 문제

다음 단계로, 큐비트에 매핑하기 전, ElectronicStructureProblem을 사용하여 페르미온 연산자 목록을 작성합니다. 여러분의 분자 시스템의 바닥 상태를 계산하기 위한 첫번째 단계입니다. 바닥 상태를 계산하는 자세한 내용은 [**this tutorial**](https://qiskit.org/documentation/nature/tutorials/03_ground_state_solvers.html)에서 확인할 수 있습니다.

<center><img src="resources/H2_gs.png" width="300"></center>

In [ ]:
from qiskit_nature.problems.second_quantization import ElectronicStructureProblem

##############################
# Provide your code here

es_problem = 

##############################
second_q_op = es_problem.second_q_ops()
print(second_q_op[0])

### 3. 큐비트 변환기(QubitConverter)

시뮬레이션에 사용할 매핑의 종류를 설정할 수 있게 해줍니다.

In [ ]:
from qiskit_nature.converters.second_quantization import QubitConverter
from qiskit_nature.mappers.second_quantization import JordanWignerMapper, ParityMapper, BravyiKitaevMapper

##############################
# Provide your code here

qubit_converter = 

##############################

qubit_op = qubit_converter.convert(second_q_op[0])
print(qubit_op)

### 4. 초기 상태

이론 부분에서 설명한 것과 같이 화학 계산에서 사용하기 좋은 초기 상태는 HF 상태입니다. 다음과 같이 초기화할 수 있습니다:

In [ ]:
from qiskit_nature.circuit.library import HartreeFock

##############################
# Provide your code here

init_state = 

##############################

init_state.draw()

### 5. 안사츠(Ansatz)

바닥 상태를 계산하기 위해서는 좋은 양자 회로를 선택하는 것이 몹시 중요합니다. 
Qiskit circuit library를 사용해 여러분만의 회로를 작성하는 예제들을 아래에서 확인해 보세요.

In [ ]:
from qiskit.circuit.library import EfficientSU2, TwoLocal, NLocal, PauliTwoDesign
from qiskit_nature.circuit.library import UCCSD, PUCCD, SUCCD

##############################
# Provide your code here

ansatz = 

##############################

ansatz.decompose().draw()

## 바닥 상태 에너지 계산

### Numpy를 사용해 계산하기

학습 목적으로, 행렬의 대각화를 통해 이 문제를 정확하게 푼 값을 VQE가 얻어야 하는 목표값으로 삼을 수 있습니다. 이 행렬의 차원은 분자 궤도의 수가 늘어남에 따라 기하급수적으로 확장되기 때문에 문제의 대상인 큰 분자의 경우 계산에 다소 시간이 소요 될수 있습니다. 아주 커다란 분자 시스템을 계산할 경우 파동 함수를 저장할 메모리 공간이 부족할 수 있습니다.

<center><img src="resources/vqe.png" width="600"></center>

In [ ]:
from qiskit.algorithms import NumPyMinimumEigensolver
from qiskit_nature.algorithms import GroundStateEigensolver

##############################
# Provide your code here

numpy_solver = 
numpy_ground_state_solver = 
numpy_results = 

##############################

exact_energy = numpy_results.computed_energies[0]
print(f"Exact electronic energy: {exact_energy:.6f} Hartree\n")
print(numpy_results)

In [ ]:
# Check your answer and submit using the following code
from qc_grader import grade_ex2c

grade_ex2c(numpy_results)

### VQE로 계산하기

다음 단계는 VQE를 사용하여 바닥 상태 에너지를 계산하는 것입니다. 여러분의 전자 문제의 해결까지 이제 절반가량 왔습니다!

In [ ]:
from qiskit.providers.aer import StatevectorSimulator, QasmSimulator
from qiskit.algorithms.optimizers import COBYLA, L_BFGS_B, SPSA, SLSQP

##############################
# Provide your code here

backend = 
optimizer = 

##############################

In [ ]:
from qiskit.algorithms import VQE
from qiskit_nature.algorithms import VQEUCCFactory, GroundStateEigensolver
from jupyterplot import ProgressPlot
import numpy as np

error_threshold = 10 # mHartree

np.random.seed(5)  # fix seed for reproducibility
initial_point = np.random.random(ansatz.num_parameters)

In [ ]:
# for live plotting
pp = ProgressPlot(plot_names=['Energy'],
                  line_names=['Runtime VQE', f'Target + {error_threshold}mH', 'Target']) 

intermediate_info = {
    'nfev': [],
    'parameters': [],
    'energy': [],
    'stddev': []
}

def callback(nfev, parameters, energy, stddev):
    intermediate_info['nfev'].append(nfev)
    intermediate_info['parameters'].append(parameters)
    intermediate_info['energy'].append(energy)
    intermediate_info['stddev'].append(stddev)
    pp.update([[energy, exact_energy+error_threshold/1000, exact_energy]])

##############################
# Provide your code here

vqe = 
vqe_ground_state_solver = 
vqe_results = 

##############################

print(vqe_results)

In [ ]:
error = (vqe_results.computed_energies[0] - exact_energy) * 1000 # mHartree
print(f'Error is: {error:.3f} mHartree')

In [ ]:
# Check your answer and submit using the following code
from qc_grader import grade_ex2d

grade_ex2d(vqe_results)

## 들뜬 상태 계산 

### QEOM을 사용한 계산

이번에는 여러분의 분자 해밀토니안의 들뜬 상태를 계산해 보도록 하겠습니다. 시스템은 위에서 이미 정의했으므로, 이번에는 양자 운동 방정식(qEOM) 알고리즘을 사용하여 들뜬 상태의 에너지를 아래의 유사 고유값 문제로 풀어보도록 하겠습니다.

<center><img src="resources/math-1.svg" width="400"></center>

에서

<center><img src="resources/math-2.svg" width="300"></center>

이며, 각 매트릭스 요소는 각각에 해당하는 기저 상태와 함께 양자 컴퓨터에서 측정되어야 합니다.

더 깊은 이해를 위해, 들뜬 상태 계산에 대한 [**this tutorial**](https://qiskit.org/documentation/nature/tutorials/04_excited_states_solvers.html)을 읽어보시길 권장하며, qEOM 이론 자체는 [**corresponding paper by Ollitrault et al., 2019**](https://arxiv.org/abs/1910.12890)을 읽어보시길 권장합니다. 

In [ ]:
from qiskit_nature.algorithms import QEOM

##############################
# Provide your code here  

qeom_excited_state_solver = 
qeom_results = 

##############################

print(qeom_results)

In [ ]:
# Check your answer and submit using the following code
from qc_grader import grade_ex2e
grade_ex2e(qeom_results)

최종적으로, 위에서 얻은 두 세트의 들뜬 상태의 에너지와 바닥 상태의 에너지 차이를 계산하여 밴드 갭 또는 에너지 갭(전자가 바닥 상태에서 들뜬 상태로 벗어나는 데 필요한 최소 에너지량)을 얻게 됩니다.

In [ ]:
bandgap = qeom_results.computed_energies[1] - qeom_results.computed_energies[0]
bandgap # in Hartree

## Qiskit  Runtime을 사용하여 클라우드에서 VQE를 실행하기

Qiskit Runtime은 IBM Quantum이 제공하는 새로운 아키텍처로, 많은 반복이 필요한 계산을 간소화합니다. Qiskit runtime은 개선된 하이브리드 양자/고전 프로세스 내에서 실험의 실행 속도를 눈에 띄게 개선합니다. 

인증된 사용자는 Qiskit Runtime을 사용해 자신과 다른 사용자가 사용할 수 있도록 Qiskit으로 작성된 양자 프로그램을 업로드할 수 있습니다. Qiskit 양자 프로그램은 Qiskit Runtime 프로그램이라고도 불리며 특정 입력을 받아 양자 및 고전적인 계산을 수행하고 원하는 경우 반응형으로 중간 결과를 제공하고 처리 결과를 반환하는 파이썬으로 작성된 코드를 의미합니다. 사용자 본인 혹은 다른 인증된 사용자는 프로그램 동작에 필요한 입력 매개 변수를 전송하여 업로드된 양자 프로그램을 호출해 사용할 수 있습니다. 


<center><img src="resources/qiskit-runtime1.gif" width="600"></center>

<center><img src="resources/runtime_arch.png" width="600"></center>


Qiskit Runtime을 사용하여 VQE를 실행하기 위해서는, 로컬 VQE 실행 단계에서의 변화는 거의 없으므로, VQE 클래스를 VQEProgram 클래스로 교체해야 합니다. 두 가지 모두 동일한 MinimumEigensolver 인터페이스를 따르기때문에 compute_minimum_eigenvalue 메서드를 공유하여 알고리즘이 실행되고 같은 유형의 결과 개체가 반환합니다. 단지 이니셜라이저의 서명이 약간 다를 뿐입니다.

먼저 Qiskit Runtime 서비스에 액세스할 수 있는 공급자와 회로를 실행할 백엔드를 선택합니다.

Qiskit Runtime에 대한 추가정보는, [**VQEProgram**](https://qiskit.org/documentation/partners/qiskit_runtime/tutorials/vqe.html#Runtime-VQE:-VQEProgram)과 [**Leveraging Qiskit Runtime**](https://qiskit.org/documentation/nature/tutorials/07_leveraging_qiskit_runtime.html) 튜토리얼을 참고 하십시오.

In [ ]:
from qc_grader.util import get_challenge_provider
provider = get_challenge_provider()
backend = provider.get_backend('ibmq_qasm_simulator')

In [ ]:
from qiskit_nature.runtime import VQEProgram

error_threshold = 10 # mHartree

# for live plotting
pp = ProgressPlot(plot_names=['Energy'],
                  line_names=['Runtime VQE', f'Target + {error_threshold}mH', 'Target']) 

intermediate_info = {
    'nfev': [],
    'parameters': [],
    'energy': [],
    'stddev': []
}

def callback(nfev, parameters, energy, stddev):
    intermediate_info['nfev'].append(nfev)
    intermediate_info['parameters'].append(parameters)
    intermediate_info['energy'].append(energy)
    intermediate_info['stddev'].append(stddev)
    pp.update([[energy,exact_energy+error_threshold/1000, exact_energy]])

##############################
# Provide your code here 

optimizer = {
    'name': 'QN-SPSA',  # leverage the Quantum Natural SPSA
    # 'name': 'SPSA',  # set to ordinary SPSA
    'maxiter': 100,
}
    
runtime_vqe = 

##############################

<div class="alert alert-block alert-success">

**도전 2f 채점 설명**
    
이 도전 과제의 채점은 이전의 연습들과 약간 다릅니다.
 
1. 우선 `prepare_ex2f`를 사용해 `runtime_vqe (VQEProgram)`, `qubit_converter (QubitConverter)`, `es_problem (ElectronicStructureProblem)`이 포함된 runtime job을 IBM Quantum으로 시뮬레이션 계산을 위해 전송합니다. 대기 상태에 따라 실행에 몇 분이 소요될 수 있습니다. 내부적으로 `prepare_ex2f`은 다음의 작업을 합니다:
```python
runtime_vqe_groundstate_solver = GroundStateEigensolver(qubit_converter, runtime_vqe)
runtime_vqe_result = runtime_vqe_groundstate_solver.solve(es_problem)
```
    
2. 작업이 완료된 후, `grade_ex2f`를 사용해 정답을 확인한 후 제출합니다.</div>

In [ ]:
# Submit a runtime job using the following code
from qc_grader import prepare_ex2f
runtime_job = prepare_ex2f(runtime_vqe, qubit_converter, es_problem)

In [ ]:
# Check your answer and submit using the following code
from qc_grader import grade_ex2f
grade_ex2f(runtime_job)

In [ ]:
print(runtime_job.result().get("eigenvalue"))

축하합니다! 첫 번째 Qiskit Runtime 프로그램을 제출하고 도전 과제를 성공했습니다.

하지만 즐거움은 끝나지 않았습니다! 여러분을 위해 양자 챌린지를 위한 전용 양자 시스템을 예약해 두었습니다. 채점되지 않는 보너스 연습으로 실제 양자 시스템에 VQE 런타임 작업을 실행해 보십시오!

<div class="alert alert-block alert-success">

** VQE를 실제 양자 시스템에서 실행시키기(부수적인)**
    
이 과제를 위해 전용 양자 시스템 [`ibm_perth`](https://quantum-computing.ibm.com/services?services=systems&system=ibm_perth)을 예약해 두었습니다. 실제 양자 시스템에 런타임 작업을 제출하려면 아래 단계를 따르십시오.
    
1. 백엔드 선택을 `ibm_perth`로 업데이트하고 `runtime_vqe`에 다시 전달하십시오.
    ```
    backend = provider.get_backend('ibm_perth')
    runtime_vqe = VQEProgram(...
                             backend=backend, 
                             ...)
    ```
2. `prepare_ex2f`의 `real_device`플래그를 `True`로 설정합니다.
3. `prepare_ex2f`를  실행하여 runtime 작업을 `ibm_perth`로 전달합니다.

</div>

<div class="alert alert-block alert-danger">

노트: Qiskit 런타임은 VQE 속도를 최대 5배까지 높입니다. 하지만 각 런타임 작업은 여전히 30~60분의 양자 프로세서 시간이 소요될 수 있습니다. 따라서 **작업** 을 완료하기 위한 대기열 시간은 작업을 제출하는 참가자 수에 따라 몇 시간 또는 며칠이 될 수도 있습니다.

모든 참가자에게 즐거운 경험을 제공하기 위해 먼저 시뮬레이터를 사용하여 다음 설정을 시도한 이후에만 실제 양자 시스템에 작업을 제출해 주십시오
1. `PartiyMapper`를 사용하고 `two_qubit_reduction=True`를 설정하여 필요한 큐비트 수를 2로 줄여서 VQE 프로그램이 바닥 상태 에너지에 더 빠르게 수렴하도록 합니다(더 적은 반복 횟수로).
1. 최적화 옵션 중 'maxiter=100 혹은 그 이하로 제한합니다. 시뮬레이터를 사용하여 최적의 낮은 반복 횟수를 찾습니다.
1. 시뮬레이터를 백엔드로 사용하여 `grade_ex2f`에 VQE를 전달하여 런타임 프로그램이 올바르게 작성 되었는지 먼저 확인합니다.
1. 더 많은 참가자가 실제 양자 시스템에서 런타임을 시도할 수 있도록 작업을 참가자당 1개로 제한합니다.
    
작업 실행이 너무 오래 걸리거나 챌린지가 종료되기 전에 실행할 수 없는 경우에도 걱정하지 마십시오. 이것은 부수적인 도전입니다. 실제 양자 시스템에서 작업을 실행하지 않고도 모든 도전 과제를 통과하고 디지털 배지를 취득 할 수 있습니다.
</div>

In [ ]:
# Please change `real_device` flag to True if you want to send a job to the real quantum system
runtime_job_real_device = prepare_ex2f(runtime_vqe, qubit_converter, es_problem, real_device=False)

In [ ]:
print(runtime_job_real_device.result().get("eigenvalue"))

## 추가 정보

**제작자:** Junye Huang, Samanvay Sharma

**한글번역:** 신소영, 김정원

**Version:** 1.0.0